# Etapa 1: Treinamento do Modelo (Fine-tuning)

**Arquitetura:**  
Utilize o YOLOv11 ou YOLOv12. Escolha uma versão do modelo adequada para cenários de borda (ex: Nano ou Small).

**Dataset:**  
Utilize o dataset público Roboflow 100: Road Signs. Escolha entre 5 e 8 das classes marcadas no dataset.  
[Link do dataset](https://universe.roboflow.com/roboflow-100/road-signs-6ih4y)

**Requisito:**  
O objetivo não é obter o mAP perfeito, mas sim gerar um modelo funcional capaz de detectar as classes do dataset em novos vídeos/imagens.  
Não é necessário esperar muitas épocas do treinamento para agilizar o processo.


In [2]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.5 MB/s eta 0:00:00


In [3]:
!pip install roboflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 26.7 MB/s eta 0:00:00


In [4]:
from roboflow import Roboflow

rf = Roboflow(api_key="N3boCmZV8VBiMKV1vmYw")
project = rf.workspace().project("road-signs-6ih4y-swarb")

dataset = project.version(3).download("yolov11")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to road-signs-3 in yolov11:: 100%|██████████| 4198/4198 [00:01<00:00, 3744.70it/s]


In [5]:
!cat  road-signs-3/data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 7
names: ['do_not_enter', 'parking', 'ped_zebra_cross', 'red_light', 'stop', 'traffic_light', 'warning']

roboflow:
  workspace: mickaelly
  project: road-signs-6ih4y-swarb
  version: 3
  license: CC BY 4.0
  url: https://universe.roboflow.com/mickaelly/road-signs-6ih4y-swarb/dataset/3

In [6]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")

model.train(
    data="road-signs-3/data.yaml",
    epochs=15,
    imgsz=640,
    batch=16,
    device=0
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=road-signs-3/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kob

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b74708fcc50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [9]:

results = model.val()

import numpy as np

print(f"mAP50-95: {results.box.map:.4f}")
print(f"mAP50: {results.box.map50:.4f}")
print(f"Precision (média): {np.mean(results.box.p):.4f}")
print(f"Recall (média): {np.mean(results.box.r):.4f}")


Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1194.6±339.3 MB/s, size: 40.1 KB)
val: Scanning /content/road-signs-3/valid/labels.cache... 488 images, 308 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 488/488 371.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 31/31 4.0it/s 7.8s
                   all        488        195      0.926      0.909      0.936      0.755
          do_not_enter         30         30      0.916      0.967      0.981      0.859
       ped_zebra_cross         30         36      0.973      0.992      0.994      0.774
             red_light         30         35      0.759      0.714      0.773      0.498
                  stop         30         34      0.967       0.85      0.935      0.763
         traffic_light         30         30      0.965      0.929      0.938       0.77
               warning         3

In [8]:

model.export(format="onnx", imgsz=640, opset=17)


import shutil
import os

os.makedirs("models", exist_ok=True)


shutil.copy("runs/detect/train/weights/best.onnx", "models/yolov11n.onnx")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 11, 8400) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 182ms
Prepared 6 packages in 7.64s
Installed 6 packages in 269ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.19.1
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.78

requirements: AutoUpdate success ✅ 8.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 17...
ONNX: slimming with onnxslim 0.1.78...
ONNX

'models/yolov11n.onnx'